### **Implicit vs Explicit Waits**

En este vídeo, voy a explicarte la diferencia entre "`waits implícitos`" y "`waits explícitos`" y también te diré cuál es el momento adecuado para utilizar cada uno de ellos. Pero primero, ¿por qué necesitamos usar "waits"? Bueno, uno de los problemas de los sitios web basados en JavaScript es que los datos se cargan dinámicamente, por lo que puede tardar unos segundos en mostrar todos los datos correctamente. Como resultado, es posible que un elemento no se localice al scrapear el sitio web. Así que obtendremos una "excepción de elemento no visible". Por eso tenemos que hacer que el driver espere hasta que los datos que queremos scrapear se carguen completamente. 

Hay dos tipos de "waits": 
- Las "esperas implícitas" y 
- las "esperas explícitas". 

Un "`wait implícito`" se utiliza para indicar al driver que espere un cierto tiempo cuando intenta localizar un elemento. En Python, puedes importar la librería "`time`" y luego hacer una espera implícita con "`time.sleep()`", y dentro de paréntesis especificas los segundos que quieres que espere el driver. Por ejemplo, si quieres una espera de dos segundos, escribes "`time.sleep(2)`"

Por otro lado, un "`wait explícito`" hace que el web driver espere a que se produzca una condición específica antes de continuar con la ejecución. Para trabajar con la "espera explícita" tenemos que importar tres librerías: `By`, `WebDriverWait` y `EC`, que significa condiciones esperadas (Expected Conditions). Después de esto, podemos crear una espera explícita. Ahora, veamos la sintaxis de una espera explícita: La espera explícita es un poco larga, así que la explico en tres partes. En la primera parte tienes que escribir "`WaitDriverWait`" y luego dentro de paréntesis pones el `driver` y luego los `segundos` que quieres que espere el driver. Luego escribes "`.until()`", dentro vamos a escribir una condición. Así que esta primera parte significa: espera hasta que esta condición suceda y espera como máximo diez segundos. Por lo tanto, si después de diez segundos la condición no se cumple, el código se va a romper. 

> **WebDriverWait(driver,10).until(...)**

Ahora, veamos la sintaxis de la condición: Así que dentro del paréntesis escriba "`EC`" que significa "Expected Condition" y luego "`.`" y lo que sigue es la condición. Hay diferentes tipos de condición, por ejemplo, "`presence of element located`" o "`element to be clickable`" o "`element to be selected`" y así sucesivamente. Tienes que elegir el que necesites. Una de las condiciones más populares es "`presence of element located`". Esta hace que el driver espere hasta que la presencia de un elemento sea localizada con éxito, de lo contrario el código se romperá.

> **WebDriverWait(driver,10).until(EC.presence_of_element_located(...))**

Finalmente, la última parte de esta sintaxis es cómo localizamos este elemento. Como recordarás, hay muchas formas de localizar un elemento: con XPath, class name, I.D y así sucesivamente. En este caso, tenemos que escribir "`By`" y luego "`.`" y luego el "`atributo`" o "`Xpath`". Luego añadimos "`,`" y escribimos el `valor` entre comillas. Si juntamos estas tres partes, obtendremos esta sintaxis completa. Puede parecer un poco aterrador, pero no te preocupes, te acostumbrarás.

> **WebDriverWait(driver,10).until(EC.presence_of_element_located(By.Xpath,"value"))**

Trabajamos sobre el sitio:

https://www.audible.com/adblbestsellers?page=5&ref_pageloadid=not_applicable&ref=a_adblbests_c5_pageNum_3&pf_rd_p=3c1c017b-585f-4bde-98c6-e3cb784e4b8e&pf_rd_r=KQEQBD63Z7NAG2RQ5J7Y&pageLoadId=2tFHu56cEAoM1VOM&creativeId=0bf0e03f-bb55-481b-b4fd-d67375977170

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Modo Headless
options = Options()  # Inicializar una instancia de la clase Options
options.headless = False  # False -> Modo Headless desactivado
# options.add_argument('window-size=1920x1080')  # Establece un tamaño de ventana grande, para que se muestren todos los datos

# El enlace del sitio web con el que trabajaremos tiene 5 páginas
# Se recomienda utilizar Chrome, pero podriamos utilizar Firefox, Safari, Edge, etc.
website = "https://www.audible.com/adblbestsellers?ref=a_search_t1_navTop_pl0cg1c0r0&pf_rd_p=adc4b13b-d074-4e1c-ac46-9f54aa53072b&pf_rd_r=1F7DV0MPHV77Z61RX566"
# website = "https://www.audible.com/search"
driver = webdriver.Chrome(executable_path = r'C:\Users\Alfonso\Downloads\Programas\chromedriver\chromedriver.exe', options = options)
driver.get(website)
driver.maximize_window()

pagination = driver.find_element_by_xpath('//ul[contains(@class, "pagingElements")]')
pages = pagination.find_elements_by_tag_name('li')
last_page = int(pages[-2].text)

book_title = []
book_author = []
book_length = []
current_page = 1

while current_page <= last_page:
    # Implicit Wait
    time.sleep(2)
    # Explicit Wait
    # container = driver.find_element_by_class_name('adbl-impression-container ')
    container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'adbl-impression-container ')))
    # products = container.find_elements_by_xpath('.//li[contains(@class, "productListItem")]')
    products = WebDriverWait(container, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/li[contains(@class, "productListItem")]')))

    for product in products:
        book_title.append(product.find_element_by_xpath('.//h3[contains(@class, "bc-heading")]').text)
        book_author.append(product.find_element_by_xpath('.//li[contains(@class, "authorLabel")]').text)
        book_length.append(product.find_element_by_xpath('.//li[contains(@class, "runtimeLabel")]').text)

    current_page = current_page + 1
    try:
        next_page = driver.find_element_by_xpath('.//span[contains(@class , "nextButton")]')
        next_page.click()
    except:
        pass

driver.quit()

df_books = pd.DataFrame({'title': book_title, 'author': book_author, 'length': book_length})
df_books.to_csv('books_pagination_2.csv', index=False)

In [3]:
df_books

,title,author,length
0,1. The Women,By: Kristin Hannah,Length: 14 hrs and 57 mins
1,2. Atomic Habits,By: James Clear,Length: 5 hrs and 35 mins
2,3. The Teacher,By: Freida McFadden,Length: 9 hrs and 35 mins
3,4. First Lie Wins,By: Ashley Elston,Length: 9 hrs and 16 mins
4,5. A Court of Thorns and Roses,By: Sarah J. Maas,Length: 16 hrs and 7 mins
...,...,...,...
95,96. Hello Beautiful,By: Ann Napolitano,Length: 15 hrs and 6 mins
96,97. Present over Perfect,By: Shauna Niequist,Length: 4 hrs and 44 mins
97,98. Ask for Andrea,By: Noelle West Ihli,Length: 6 hrs and 40 mins
98,99. The Chronicles of Narnia Complete Audio Co...,By: C. S. Lewis,Length: 33 hrs and 45 mins
